In [1]:
import pandas as pd
import numpy as np
import plotly
import plotly.express as px

import scipy
from matplotlib.collections import LineCollection
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [3]:
#comparisons = pd.DataFrame("", columns=['contrast'])
metadata = pd.read_csv("../../config/ALLcoldata.txt", sep="\t")
meta = metadata[['condition', 'species']].drop_duplicates()
meta.loc[:,'condition'] = meta['condition'].str.replace("_fun", "")
# Read in .csv file containing selection signals and associated metadata
pfam_df = pd.read_csv("../../../rna-seq-busia/resources/Anogam_long.pep_Pfamscan.seqs", sep="\s+", header=None)
go_df = pd.read_csv("../../../rna-seq-busia/resources/Anogam_long.pep_eggnog_diamond.emapper.annotations.GO", sep="\t", header=None)

pfam_df.columns = ["transcript", "pstart", "pend", "pfamid", "domain", "domseq"]
go_df.columns = ['transcript', 'GO_terms']

gene_annot_df = pfam_df.merge(go_df)
gene_annot_df.loc[:, 'gene_id'] = gene_annot_df.loc[:, 'transcript'].str.replace("Anogam_", "").str.replace("-R[A-Z]", "")

/home/sanj/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]


In [4]:
def gene_ids_from_domain(gene_annot_df, domain):
    gene_list = []
    if isinstance(domain, list):
        for dom in domain:
            ids = gene_annot_df.query("domain == @domain")['gene_id']
            gene_list.append(ids)
            return(np.unique(gene_list))
    else:
        return(gene_annot_df.query("domain == @domain")['gene_id'].to_numpy())

def load_species(df, meta):
    spp=np.array([])
    for c in df['comparison']:
        c = c.split("_")[0]
        spps = meta.query(f"condition == '{c}'")['species'].unique()
        if len(spps) == 0: print(c)
        if len(spps) > 1: print(len(spps))
        spp = np.append(spp, spps)
    return spp

In [12]:
### MAIN
# a dict with gene families and their respective Pfam domain for extracting
gene_fams = {'CSP': 'OS-D',
                    'Cytochrome P450s':'p450', 
             'GSTs':['GST_N', 'GST_N_3', 'GST_C'], 
             'ABC-transporters':['ABC_membrane', 'ABC_tran'],
            'Carboxylesterases': 'COesterase', 
             'Odorant binding proteins':'PBP_GOBP', 
             'Olfactory receptors':'7tm_6', 
             'Ionotropic receptors':['Lig_chan','7tm_1'],
             'Gustatory receptors': '7tm_7',
            'Fatty acid synthases':'ketoacyl-synt',
            'FA Elongase':'ELO',
            'FA desaturase':'FA_desaturase',
            'FA reductase':'NAD_binding_4'}


fc_data = pd.read_csv("../../results/fc_data.tsv", sep="\t")
pval_data = pd.read_csv("../../results/pval_data.tsv", sep="\t")
assert (fc_data['GeneID'] == pval_data['GeneID']).all()

#names = fc_data[['GeneID', 'GeneName']]
fc_data = fc_data.iloc[:, :-1]

AssertionError: 

In [6]:
   def plotly_strip_genes(fc_data, gene_ids, meta):
        
    fam_fc_data = fc_data.query("GeneID in @gene_ids").copy()
    fam_fc_data.loc[:, 'Label'] = [id_ + " | " + name if name != "" else id_ for id_, name in zip(fam_fc_data['GeneID'].fillna(""), fam_fc_data['GeneName'].fillna(""))]
    fam_fc_data =fam_fc_data.drop(columns=['GeneName', 'GeneID']).melt(id_vars='Label', var_name='comparison', value_name='log2FC')
    fam_fc_data.loc[:, 'comparison'] = fam_fc_data['comparison'].str.replace("_log2FoldChange", "")
    fam_fc_data.loc[:, 'species'] = load_species(fam_fc_data, meta)

    fig = px.strip(
        fam_fc_data, 
        y='Label', 
        x='log2FC', 
        color='species',
        title=name, 
        hover_data=['comparison'],
        width=1000, 
        height=2000,
        template='ggplot2'
    )
    fig.update_layout(titlefont=dict(size =20, color='black', family='Arial, sans-serif'), xaxis_range=[-4,6])
    fig.add_vline(0,  line_width=3, line_dash="dash", line_color="grey")
    fig.show()

In [ ]:
for name, domain in gene_fams.items():
    
    gene_ids = gene_ids_from_domain(gene_annot_df, domain)
    plotly_strip_genes(fc_data, gene_ids, meta)